In [1]:
import pandas as pd
import numpy as np
import math as m
import itertools
from collections import defaultdict
from pychattr.channel_attribution import PyChAttr
from rpy2.robjects.packages import importr
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 2000)
Rbase = importr("base")
RChannelAttribution = importr("ChannelAttribution")
import warnings
warnings.simplefilter('ignore')

In [2]:
"""
1493 - Kasko_formalize - при нажатии на кнопку "Kasko_оформить"
1497 - Health_life_DMS_send_request - при нажатии на кнопку "Health_life_DMS_отправить заявку"
1504 - Property_House_buy_online - при нажатии на кнопку "Property_отправить"
1527 - travel payment - переход на последний шаг калькулятора 
1534 - Incident_Payment - несчастный случай_Оплата
1536 - Flat payment - переход на последний шаг калькулятора 
1537 - Mortgage Payment - на кнопку оформление и получение полиса 
1538 - Osago Payment - переход на последний шаг калькулятора 
1540 - go payment - переход на последний шаг калькулятора 
""";

In [3]:
df1493 = pd.read_csv('prepared_data1493.csv')
df1497 = pd.read_csv('prepared_data1497.csv')
df1504 = pd.read_csv('prepared_data1504.csv')
df1527 = pd.read_csv('prepared_data1527.csv')
df1534 = pd.read_csv('prepared_data1534.csv')
df1536 = pd.read_csv('prepared_data1536.csv')
df1537 = pd.read_csv('prepared_data1537.csv')
df1538 = pd.read_csv('prepared_data1538.csv')
df1540 = pd.read_csv('prepared_data1540.csv')
frames = [df1493, df1497, df1504, df1527, df1534, df1536, df1537, df1538, df1540]
df = pd.concat(frames).reset_index(drop=True)
df = df.drop_duplicates(subset=['customer_id', 'date'])

In [31]:
target = "sum"
target_id = str(target)+"/"
csv = "prepared_data"+str(target)+".csv"
names_for_save = [target_id+"avg_conv_time"+str(target)+".csv",target_id+"transition_m"+str(target)+".csv",target_id+"removal_effect"+str(target)+".csv",
                  target_id+"attr_all"+str(target)+".csv",target_id+"attr_multi"+str(target)+".csv",target_id+"conv"+str(target)+".csv",
                 target_id+"index_cost_all"+str(target)+".csv",target_id+"index_cost_multi"+str(target)+".csv", target_id+"freq_chain"+str(target)+".csv",
                 target_id+"time_conv"+str(target)+".csv", target_id+"time_conversion"+str(target)+".csv", target_id+"one_chain_conv"+str(target)+".csv"]
#names_for_save

In [29]:
df = pd.read_csv(csv)

#print(df.shape)
#df

FileNotFoundError: File b'prepared_datasum.csv' does not exist

In [32]:
channels = set(df["channel"].unique())

In [33]:
channels

{'Adv',
 'Buzzoola',
 'DBM',
 'Mytarget_mail',
 'Mytarget_video',
 'Nativeroll',
 'OTM',
 'Rbc',
 'Soloway',
 'Vedomosti'}

In [4]:
conv_id_goal = [1493,1497,1504,1527,1529,1534,1536,1537,1538,1540]

In [5]:
k = {0:0,
     1514:0,
     1473:0,
     1478:0,
     1486:0,
     1484:0,
     1477:0,
     1511:0,
     1512:0,
     1468:0,
     1463:0,
     1472:0,
     1510:0,
     1519:0,
     1518:0,
     1521:0,
     1520:0,
     1485:0,
     1523:0,
     1525:0,
     1524:0,
     1522:0,
     1476:0,
     1498:0,
     1499:0,
     1466:0,
     1497:0,
     1496:0,
     1532:0,
     1533:0,
     1534:0,
     1531:0,
     1469:0,
     1471:0,
     1492:0,
     1465:0,
     1493:0,
     1494:0,
     1526:0,
     1495:0,
     1491:0,
     1481:0,
     1480:0,
     1517:0,
     1516:0,
     1515:0,
     1474:0,
     1541:0,
     1509:0,
     1508:0,
     1507:0,
     1467:0,
     1542:0,
     1475:0,
     1504:0,
     1505:0,
     1506:0,
     1503:0,
     1482:0,
     1502:0,
     1500:0,
     1513:0,
     1483:0,
     1464:0,
     1489:0,
     1488:0,
     1527:0,
     1490:0,
     1487:0,
     1479:0,
     1460:0,
     1461:0,
     1462:0,
     1459:0,
     1470:0,
     1501:0,
     99901:0,
     1536:0, 
     1537:0, 
     1538:0, 
     1528:0,
     1529:0,
     1530:0,
     1535:0,
     1539:0,
     1540:0
    }
#k[target]=1
for target in conv_id_goal:
    k[target] = 1

In [6]:
df["conversion"] = df["Conversion page ID"].map(k)
#print(df["conversion"].unique())
#print(df["Conversion page ID"].unique())


In [20]:
df.groupby('customer_id').max()#.sort_values(['customer_id','date'],ascending=[False, True])#

,Unnamed: 0,date,channel,conversion,Conversion page ID,Insertion name,Conversion page name,Conversion type,Action type
customer_id,,,,,,,,,
-07xkwNdKZf-,14,2018-07-26 04:38:53,DBM,1,1538,dbm_retarg_kasko,[Ingos] visitors 60 sec,impression,tracker
-5ED7rfyHTv-,15,2018-06-22 14:29:07,Mytarget_video,1,1527,msk_video_3,[Ingos] visitors 60 sec,event,tracker
-@yv8jFpS2L-,46,2018-06-09 22:22:47,Mytarget_video,1,1527,msk_video_1,[Ingos] visitors 60 sec,event,tracker
-AdaU5c7Cjj8,19,2018-06-19 20:52:19,Buzzoola,1,1497,msk_video_1,[Ingos] visitors 60 sec,event,tracker
-HrapAU79c38,71,2018-07-25 11:24:55,Nativeroll,1,1527,nativeroll_msk_video_2,[Ingos] visitors 60 sec,event,tracker
-Mqylqmyw2D8,35,2018-06-26 13:54:06,Buzzoola,1,1538,msk_video_2,[Ingos] visitors 60 sec,event,tracker
-OGWhLfk-578,42,2018-06-24 10:01:55,Mytarget_mail,1,1538,msk_kasko_1,[Ingos] visitors 60 sec,impression,tracker
-TUEvKefki79,38,2018-06-15 17:36:23,Nativeroll,1,99901,mobile_msk_video_2,[Ingosstrah] Ingosstrah_Shop_Buy_Online_Button,impression,tracker
-TnXIS4OBq-9,91,2018-06-18 07:33:15,Nativeroll,1,1527,spb_flat_calc,[Ingos] visitors 60 sec,event,tracker


In [37]:
df_ = df.copy()

In [38]:
#sum(df["conversion"].isnull())

In [39]:
df = df[["customer_id","date","channel","conversion"]]
#df["conversion"].sum()

In [40]:
df = df.sort_values(["customer_id", "date"]).reset_index(drop=True)
df["path_no"] = df.groupby("customer_id")["conversion"].shift(1).fillna(0)#.cumsum()#.shift(1).fillna(0)
df["path_no"] = df.groupby("customer_id")["path_no"].cumsum()+ 1
df["path_no"] = df["path_no"].astype(int)
#df

In [41]:
df[df["path_no"]==1]["conversion"].sum()

1563

### df_paths_1 оставляем только первую конверсию

In [42]:
#1st purchase
df_paths_1 = df[df["path_no"]==1].loc[:,"customer_id":"conversion"]
#df_paths_1.head()
#df_paths_1["conversion"].sum()

In [43]:
cookie = df_paths_1.groupby("customer_id").sum().reset_index()
cookie = pd.DataFrame(cookie["customer_id"])
cookie.to_csv(target_id+"cookie"+str(target)+".csv")
cookie

,customer_id
0,-07xkwNdKZf-
1,-5ED7rfyHTv-
2,-@yv8jFpS2L-
3,-AdaU5c7Cjj8
4,-HrapAU79c38
5,-Mqylqmyw2D8
6,-OGWhLfk-578
7,-TUEvKefki79
8,-TnXIS4OBq-9
9,-Uo5D2NXxDX9


### df_path_1_uniq - однозвеньевые цепочки (Ex. (start) > VK > VK > VK > (conv) )

In [44]:
cut = df_paths_1.groupby("customer_id")["channel"].unique().apply(lambda x: 1 if len(x)==1 else 0)
df_path_1_uniq = df_paths_1[df_paths_1["customer_id"].isin((cut[cut==1].index))].reset_index(drop=True)
#df_path_1_uniq.head()

### df_path_1_multi - многозвеньевые цепочки (Ex. (start) > VK > GDN > VK > (conv) )

In [45]:
df_path_1_multi = df_paths_1[df_paths_1["customer_id"].isin((cut[cut==0].index))].reset_index(drop=True)
#df_path_1_multi.head()

In [46]:
def agg_f(x):
     return pd.Series(dict(Sequence = "%s" %  ' > '.join(x['channel'])))

### df_uniq_paths - однозвеньевые цепочки с преобразованным path : Ex. VK > VK

In [47]:
channels_subset = df_path_1_uniq.groupby('customer_id').apply(agg_f)
conversion = df_path_1_uniq.groupby('customer_id').sum()
conversion.rename(columns={"channel":"path"}, inplace=True)
conversion["path"] = channels_subset
df_uniq_paths = conversion.reset_index()
df_uniq_paths = df_uniq_paths[["customer_id", "path", "conversion"]]
channels_subset, conversion = 0, 0
#df_uniq_paths.head()

### df_multi_paths - многозвеньевые цепочки с преобразованным столбцом path : Ex. VK > GDN > VK

In [48]:
channels_subset = df_path_1_multi.groupby('customer_id').apply(agg_f)
conversion = df_path_1_multi.groupby('customer_id').sum()
conversion.rename(columns={"channel":"path"}, inplace=True)
conversion["path"] = channels_subset
df_multi_paths = conversion.reset_index()
df_multi_paths = df_multi_paths[["customer_id", "path", "conversion"]]
channels_subset, conversion = 0, 0
#df_multi_paths

### df_all_paths - все цепочки с преобразованным столбцом path

In [49]:
channels_subset = df_paths_1.groupby('customer_id').apply(agg_f)
conversion = df_paths_1.groupby('customer_id').sum()
conversion.rename(columns={"channel":"path"}, inplace=True)
conversion["path"] = channels_subset
df_all_paths = conversion.reset_index()
df_all_paths = df_all_paths[["customer_id", "path", "conversion"]]
channels_subset, conversion = 0, 0
#df_all_paths

## Heuristic models: First Touch, Last Touch, Linear Touch

### однозвеньевые цепочки

In [50]:
model_uniq_paths = PyChAttr(df=df_uniq_paths,
                          markov_model=True,
                          heuristic_model=True,
                          first_touch=True,
                          last_touch=True,
                          linear_touch=True,
                          path_feature="path",
                          conversion_feature="conversion",
                          separator=">",
                          order=1,
                          n_simulations=None,
                          max_step=None,
                          return_transition_probs=True,
                          random_state=26,
                          return_plot_data=True)
model_uniq_paths.fit()
h_uniq_paths = model_uniq_paths.heuristic_
#h_uniq_paths

In [51]:
one_chain_conv = h_uniq_paths[["channel name", "first touch"]]
one_chain_conv.rename(columns={"first touch":"conversion"}, inplace=True)
one_chain_conv["conversion"] = one_chain_conv["conversion"].astype(int)
one_chain_conv

,channel name,conversion
0,DBM,78
1,Mytarget_video,64
2,Buzzoola,183
3,Mytarget_mail,305
4,Rbc,12
5,Nativeroll,229
6,Adv,237
7,Vedomosti,12
8,Soloway,2
9,OTM,1


### многозвеньевые цепочки

In [52]:
model_multi_paths = PyChAttr(df=df_multi_paths,
                          markov_model=True,
                          heuristic_model=True,
                          first_touch=True,
                          last_touch=True,
                          linear_touch=True,
                          path_feature="path",
                          conversion_feature="conversion",
                          separator=">",
                          order=1,
                          n_simulations=None,
                          max_step=None,
                          return_transition_probs=True,
                          random_state=26,
                          return_plot_data=True)
model_multi_paths.fit()
h_multi_paths = model_multi_paths.heuristic_
#h_multi_paths

### однозвеньевые и многозвеньевые цепочки

In [53]:
model_all_paths = PyChAttr(df=df_all_paths,
                          markov_model=True,
                          heuristic_model=True,
                          first_touch=True,
                          last_touch=True,
                          linear_touch=True,
                          path_feature="path",
                          conversion_feature="conversion",
                          separator=">",
                          order=1,
                          n_simulations=None,
                          max_step=None,
                          return_transition_probs=True,
                          random_state=26,
                          return_plot_data=True)
model_all_paths.fit()
h_all_paths = model_all_paths.heuristic_
#h_all_paths

## Markov chain model

### Строится отдельно для многозвеньевых цепочек, затем складывается с результатами однозвеньевых

In [54]:
markov_multi_paths = model_multi_paths.markov_
#markov_multi_paths

In [55]:
df_sum_path_1_uniq = df_path_1_uniq[df_path_1_uniq["conversion"]==1].groupby("channel").sum().reset_index()
df_sum_path_1_uniq.rename(columns={"channel":"channel name"}, inplace=True)
#df_sum_path_1_uniq

In [56]:
df_sum_markov = markov_multi_paths.join(df_sum_path_1_uniq.set_index('channel name'), on='channel name', how='outer').fillna(0)
#df_sum_markov

In [57]:
df_sum_markov["Markov model"] = df_sum_markov["conversion"] + df_sum_markov["markov model"]
#df_sum_markov

In [58]:
df_sum_markov["markov model"].sum()

440.0

In [59]:
transition_matrix = model_multi_paths.transition_matrix_
transition_matrix = np.round(transition_matrix,2)
removal_effects = model_multi_paths.removal_effects_
removal_effects = np.round(removal_effects,2)

In [60]:
#transition_matrix

In [61]:
#removal_effects

In [62]:
df_paths_1.head()

,customer_id,date,channel,conversion
0,-07xkwNdKZf-,2018-07-26 03:35:18,DBM,0
1,-07xkwNdKZf-,2018-07-26 03:40:38,DBM,0
2,-07xkwNdKZf-,2018-07-26 03:42:43,DBM,0
3,-07xkwNdKZf-,2018-07-26 03:43:28,DBM,0
4,-07xkwNdKZf-,2018-07-26 03:43:48,DBM,0


## Game Theory model

In [63]:
def agg_f(x):
     return pd.Series(dict(Sequence = "%s" %  ','.join(x['channel'].unique())))

In [64]:
channels_subset = df_paths_1.groupby('customer_id').apply(agg_f)
conversion = df_paths_1.groupby('customer_id').sum()
conversion["channels_subset"] = channels_subset
df_channels = conversion.reset_index()
df_channels = df_channels.groupby("channels_subset").sum().reset_index()
#df_channels.sort_values("conversion",ascending=False)

In [65]:
channels_subset = df_paths_1.groupby('customer_id').apply(agg_f)
conversion = df_paths_1.groupby('customer_id').sum()
conversion["channels_subset"] = channels_subset
df_channels = conversion.reset_index()
for i in range(len(df_channels['channels_subset'])):
    df_channels['channels_subset'][i] = list(df_channels['channels_subset'][i].split(','))
    df_channels['channels_subset'][i] = sorted(df_channels['channels_subset'][i])
    df_channels['channels_subset'][i] = ','.join(df_channels['channels_subset'][i])
df_channels = df_channels.groupby("channels_subset").sum().reset_index()
#df_channels

In [66]:
def v_function(A,C_values):
    '''
    This function computes the worth of each coalition.
    inputs:
            - A : a coalition of channels.
            - C_values : A dictionnary containing the number of conversions that each subset of channels has yielded.
    '''
    subsets_of_A = subsets(A.split(","))
    worth_of_A=0
    for subset in subsets_of_A:
        if subset in C_values:
            worth_of_A += C_values[subset]
    return worth_of_A

def subsets(s):
    '''
    This function returns all the possible subsets of a set of channels.
    input :
            - s: a set of channels.
    '''
    if len(s)==1:
        return s
    else:
        sub_channels=[]
        for i in range(1,len(s)+1):
            sub_channels.extend(map(list,itertools.combinations(s, i)))
    k = list(map(",".join,map(sorted,sub_channels)))#map(",".join,map(sorted,sub_channels))
    return k

In [67]:
# First, let's convert the dataframe "subsets_conversions" into a dictionnary
C_values = df_channels.set_index("channels_subset").to_dict()["conversion"]
C_values

#For each possible combination of channels A, we compute the total number of conversions yielded by every subset of A. 
# Example : if A = {c1,c2}, then v(A) = C({c1}) + C({c2}) + C({c1,c2})


v_values = {}
for A in subsets(channels):
    v_values[A] = v_function(A,C_values)

In [68]:
n=len(channels)
shapley_values = defaultdict(int)
for channel in channels:
    for A in v_values.keys():
        if channel not in A.split(","):
            cardinal_A=len(A.split(","))
            A_with_channel = A.split(",")
            A_with_channel.append(channel)
            A_with_channel=",".join(sorted(A_with_channel))
            shapley_values[channel] += (v_values[A_with_channel]-v_values[A])*(m.factorial(cardinal_A)*m.factorial(n-cardinal_A-1)/m.factorial(n))
# Add the term corresponding to the empty set
    shapley_values[channel]+= v_values[channel]/n
    
#for channel in channels:
#    for A in v_values.keys():
#        if channel not in A.split(","):
#            cardinal_A=len(A.split(","))
#            A_with_channel = A.split(",")
#            A_with_channel.append(channel)
#            A_with_channel=",".join(sorted(A_with_channel))
#            shapley_values[channel] += (v_values[A_with_channel]-v_values[A])/(cardinal_A+1)#

In [43]:
chann = []
value = []
for w in sorted(C_values, key=C_values.get, reverse=True):
    #print(w, C_values[w])
    chann.append(w)
    value.append(C_values[w])
chains_worth = pd.DataFrame({'group_channel':chann, 'value':value})
chains_worth.to_csv(target_id+"chains_worth"+str(target)+".csv")

In [44]:
df_GT_all = pd.Series(shapley_values, name='GT model')
df_GT_all.index.name = 'channel name'
df_GT_all = pd.DataFrame(df_GT_all).reset_index()
#df_GT_all.sort_values("channel name")

In [45]:
channels_subset = df_path_1_multi.groupby('customer_id').apply(agg_f)
conversion = df_path_1_multi.groupby('customer_id').sum()
conversion["channels_subset"] = channels_subset
df_channels = conversion.reset_index()
df_channels = df_channels.groupby("channels_subset").sum().reset_index()
#df_channels.sort_values("conversion",ascending=False)

In [46]:
channels_subset = df_path_1_multi.groupby('customer_id').apply(agg_f)
conversion = df_path_1_multi.groupby('customer_id').sum()
conversion["channels_subset"] = channels_subset
df_channels = conversion.reset_index()
for i in range(len(df_channels['channels_subset'])):
    df_channels['channels_subset'][i] = list(df_channels['channels_subset'][i].split(','))
    df_channels['channels_subset'][i] = sorted(df_channels['channels_subset'][i])
    df_channels['channels_subset'][i] = ','.join(df_channels['channels_subset'][i])
df_channels = df_channels.groupby("channels_subset").sum().reset_index()
# First, let's convert the dataframe "subsets_conversions" into a dictionnary
C_values = df_channels.set_index("channels_subset").to_dict()["conversion"]
C_values

#For each possible combination of channels A, we compute the total number of conversions yielded by every subset of A. 
# Example : if A = {c1,c2}, then v(A) = C({c1}) + C({c2}) + C({c1,c2})


v_values = {}
for A in subsets(channels):
    v_values[A] = v_function(A,C_values)
n=len(channels)
shapley_values = defaultdict(int)
for channel in channels:
    for A in v_values.keys():
        if channel not in A.split(","):
            cardinal_A=len(A.split(","))
            A_with_channel = A.split(",")
            A_with_channel.append(channel)
            A_with_channel=",".join(sorted(A_with_channel))
            shapley_values[channel] += (v_values[A_with_channel]-v_values[A])*(m.factorial(cardinal_A)*m.factorial(n-cardinal_A-1)/m.factorial(n))
# Add the term corresponding to the empty set
    shapley_values[channel]+= v_values[channel]/n
df_GT_model = pd.Series(shapley_values, name='GT model')
df_GT_model.index.name = 'channel name'
df_GT_model = pd.DataFrame(df_GT_model).reset_index()
#df_GT_model.sort_values("channel name")

In [47]:
C_values

{'DBM,Mytarget_mail': 1}

## Результаты

In [1709]:
zz = pd.DataFrame({'channel name' : ['Adv',
 'Buzzoola',
 'DBM',
 'Mytarget_mail',
 'Mytarget_video',
 'Nativeroll',
 'OTM',
 'Rbc',
 'Soloway',
 'Vedomosti']})

In [1710]:
result_all_paths = zz.join(h_all_paths.set_index('channel name'), on='channel name',how='left').fillna(0)
result_all_paths = result_all_paths.join(df_sum_markov.set_index('channel name'), on='channel name',how='left').fillna(0)
result_all_paths = result_all_paths.join(df_GT_all.set_index('channel name'), on='channel name', how='left').fillna(0)
result_all_paths.drop(["conversion","markov model"], inplace=True, axis=1)
result_all_paths = result_all_paths.sort_values("channel name").reset_index(drop=True)
result_all_paths = result_all_paths.round(2)
#result_all_paths

In [1711]:
#print(result_all_paths["Markov model"].sum())
#print(result_all_paths["GT model"].sum())
#print(result_all_paths["first touch"].sum())

In [1712]:
result_multi_paths = zz.join(h_multi_paths.set_index('channel name'), on='channel name',how='left').fillna(0)
result_multi_paths = result_multi_paths.join(df_sum_markov.set_index('channel name'), on='channel name', how='left').fillna(0)
result_multi_paths = result_multi_paths.join(df_GT_model.set_index('channel name'), on='channel name', how='left').fillna(0).sort_values("channel name").reset_index(drop=True)
result_multi_paths.drop(["conversion","Markov model"], inplace=True, axis=1)
result_multi_paths = result_multi_paths.sort_values(by="channel name").reset_index(drop=True)
result_multi_paths = result_multi_paths.round(2)
#result_multi_paths

In [1713]:
#print(result_multi_paths["markov model"].sum())
#print(result_multi_paths["GT model"].sum())
#print(result_multi_paths["first touch"].sum())

In [1714]:
#def sum_same(x):
#    x = list(df_channels['channels_subset'][i].split(','))
#    x = sorted(x)
#    x = ','.join(x)
#    return x

In [1715]:
result_all_paths

,channel name,first touch,last touch,linear touch,Markov model,GT model
0,Adv,381.0,280.0,309.79,335.67,361.07
1,Buzzoola,221.0,276.0,265.57,247.38,238.15
2,DBM,82.0,102.0,96.83,94.05,93.28
3,Mytarget_mail,383.0,399.0,400.46,394.62,401.98
4,Mytarget_video,127.0,138.0,129.75,129.71,115.40
5,Nativeroll,302.0,331.0,320.63,308.75,302.03
6,OTM,1.0,1.0,1.00,1.00,1.00
7,Rbc,18.0,22.0,20.36,20.25,19.50
8,Soloway,2.0,2.0,2.00,2.00,2.00
9,Vedomosti,46.0,12.0,16.62,29.58,28.58


In [1716]:
chn_cost = pd.DataFrame({'channel name':["Buzzoola","DBM","Mytarget_mail","Vedomosti", "Adv", "Mytarget_video"],
                         'Impressions':[10495535,1160392,21328294,462485,34217811,2824175],
                        'CPM':[180, 150, 49, 1349, 56, 314]})

In [1717]:
index_cost_all = chn_cost.join(result_all_paths.set_index('channel name'), on='channel name', how='left').fillna(0)
index_cost_all = index_cost_all[["channel name", "Impressions", "CPM", "last touch", "GT model"]]
index_cost_all["LT cost index"] = index_cost_all["Impressions"]*index_cost_all["CPM"]/1000/index_cost_all["last touch"]
index_cost_all["GTm cost index"] = index_cost_all["Impressions"]*index_cost_all["CPM"]/1000/index_cost_all["GT model"]
#index_cost_all["difference,%"] = (index_cost_all["LT cost index"] - index_cost_all["GTm cost index"])/index_cost_all["LT cost index"]*100
index_cost_all.replace([np.inf, -np.inf], 0, inplace=True)
index_cost_all = index_cost_all.round(1)
index_cost_all['last touch'] = index_cost_all['last touch']
index_cost_all['GT model'] = index_cost_all['GT model']
index_cost_all['LT cost index'] = index_cost_all['LT cost index'].round(0).astype(int)
index_cost_all['GTm cost index'] = index_cost_all['GTm cost index'].round(0).astype(int)
#index_cost_all

In [1718]:
avg_time_conv = df_paths_1.groupby("customer_id")["date"].agg(['first','last'])
avg_time_conv['first'] = pd.to_datetime(avg_time_conv['first'])
avg_time_conv['last'] = pd.to_datetime(avg_time_conv['last'])
avg_time_conv["conversion_time"] = avg_time_conv["last"]-avg_time_conv["first"]
#print(avg_time_conv["conversion_time"].mean())
#print(avg_time_conv["conversion_time"].median())
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic
time_conversion = list(avg_time_conv["conversion_time"])
for i in range(len(time_conversion)):
    time_conversion[i] = int(str(time_conversion[i])[:2])+1
n = len(time_conversion)
mean = np.mean(time_conversion)
std = np.std(time_conversion,ddof=1)/np.sqrt(n)
answer = _tconfint_generic(mean, std, n - 1, 0.05, 'two-sided')
avg_time_conv.sort_values("conversion_time",ascending=False,inplace=True)

In [1719]:
time_conversion = list(avg_time_conv["conversion_time"])
for i in range(len(time_conversion)):
    time_conversion[i] = str(time_conversion[i])[:2]
time_conversion = pd.DataFrame(time_conversion,columns=['day'])
time_conversion['count'] = time_conversion['day']
time_conversion['day'] = time_conversion['day'].astype(int)
time_conversion = time_conversion.groupby('day').count().sort_values("day",ascending=False).reset_index()

In [1720]:
def agg_f(x):
     return pd.Series(dict(Sequence = "%s" %  ' > '.join(x['channel'].unique())))
channels_subset = df_path_1_multi.groupby('customer_id').apply(agg_f)
conversion = df_path_1_multi.groupby('customer_id').sum()
conversion["channels_subset"] = channels_subset
df_channels = conversion.reset_index()
df_channels = df_channels.groupby("channels_subset").sum().reset_index()
freq_chain = df_channels.sort_values("conversion",ascending=False).reset_index(drop=True)
#print(int(freq_chain["conversion"].sum()))

In [1721]:
channels_subset = df_path_1_uniq.groupby('customer_id').apply(agg_f)
conversion = df_path_1_uniq.groupby('customer_id').sum()
conversion.rename(columns={"channel":"path"}, inplace=True)
conversion["path"] = channels_subset
df_uniq_paths = conversion.reset_index()
df_uniq_paths = df_uniq_paths[["customer_id", "path", "conversion"]]
#print(int(conversion.groupby("path").sum().sum()))
#print(df_paths_1["conversion"].sum())
#print(df["conversion"].sum())
#print(df["conversion"].sum()-df_paths_1["conversion"].sum())

In [1722]:
def YMD(x):
    x = str(x.year)+"-"+str(x.month)+"-"+str(x.day)
    return x
df_['date'] = pd.to_datetime(df_['date'])
df_['date_new'] = df_['date'].apply(YMD)
df_['date_new'] = pd.to_datetime(df_['date_new'])
#time_conv = pd.DataFrame(df_[df_["Conversion page ID"]==target].groupby("date_new")["conversion"].count()).reset_index()
#time_conv conv_id_goal
time_conv = pd.DataFrame(df_[df_["Conversion page ID"].isin(conv_id_goal)].groupby("date_new")["conversion"].count()).reset_index()

In [1723]:
!mkdir $target_id
avg_time_conv.to_csv(names_for_save[0])
transition_matrix.to_csv(names_for_save[1])
removal_effects.to_csv(names_for_save[2])
result_all_paths.to_csv(names_for_save[3])
result_multi_paths.to_csv(names_for_save[4])
index_cost_all.to_csv(names_for_save[6])
freq_chain.to_csv(names_for_save[8])
time_conv.to_csv(names_for_save[9])
time_conversion.to_csv(names_for_save[10])
one_chain_conv.to_csv(names_for_save[11])

mkdir: невозможно создать каталог «sum/»: Файл существует


In [1724]:
with open(names_for_save[0],'a') as f:
    f.write(",,,95% доверительный интервал для среднего:  " + str(np.round(answer,2)) + "\n")

In [1725]:
with open(names_for_save[5], 'w') as f:
    f.write(",,,Всего конверсий: "+str(df["conversion"].sum())+"\n")
    f.write(",,,Всего первых конверсий: "+str(df_paths_1["conversion"].sum())+"\n")
    f.write(",,,Всего вторых конверсий: "+str(df[df["path_no"]==2]["conversion"].sum())+"\n")
    f.write(",,,Всего третьих конверсий: "+str(df[df["path_no"]==3]["conversion"].sum())+"\n")
    f.write(",,,Всего четвертых конверсий: "+str(df[df["path_no"]==4]["conversion"].sum())+"\n")
    f.write(",,,Всего пятых конверсий: "+str(df[df["path_no"]==5]["conversion"].sum())+"\n")
    f.write(",,,Всего шестых конверсий: "+str(df[df["path_no"]==6]["conversion"].sum())+"\n")

In [1726]:
with open(names_for_save[8],'a') as f:
    f.write(",,,Всего конверсий: "+str(df["conversion"].sum())+"\n")
    f.write(",,,Первых многозвеньевых конверсий: "+str(int(freq_chain["conversion"].sum()))+"\n")
    f.write(",,,Первых однозвеньевых конверсий: "+str(int(conversion.groupby("path").sum().sum()))+"\n")
    f.write(",,,Всего первых конверсий: "+str(df_paths_1["conversion"].sum())+"\n")
    f.write(",,,Всего вторых третьих и т.д. конверсий: "+str(df["conversion"].sum()-df_paths_1["conversion"].sum())+"\n")
    